In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nest
import numpy as np
import os

from pynestml.codegeneration.nest_code_generator_utils import NESTCodeGeneratorUtils

# NEST installation directory path
NEST_SIMULATOR_INSTALL_LOCATION = nest.ll_api.sli_func("statusdict/prefix ::")


### NESTML code generation

Assume that our NESTML input model is called `iaf_psc_exp_multisyn_exc.nestml`. To generate code and build a dynamic library that can be loaded as a user module in NEST Simulator:

In [ ]:
# Get the absolute path to the parent directory of the current working directory
repo_parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Get the absolute path to the main directory of the repository
repo_main_dir = os.path.abspath(os.path.join(repo_parent_dir, os.pardir))

# Get the absolute path to the NESTML model
nestml_model_dir = os.path.join(repo_main_dir, "src" , "nestml_models", "iaf_psc_exp_multisyn_exc_neuron.nestml")

print(nestml_model_dir)

In [ ]:
# generate_target(input_path=nestml_model_dir,
#                 target_platform="NEST",
#                 target_path="/tmp/nestml-component",
#                 logging_level="INFO"
#                 )

module_name, neuron_model_name = NESTCodeGeneratorUtils.generate_code_for(nestml_model_dir)

Check the generated log output for any potential error messages or warnings.

The generated module is called ``nestmlmodule`` by default. It can be loaded using ``nest.Install()``:

In [ ]:
nest.Install(module_name)

### Instantiate model in NEST Simulator and run

Using the PyNEST API, the model can be instantiated and simulated in NEST. The following code will create one instance of the neuron model (`nest.Create("iaf_psc_exp_multisynex")`), inject a pulse current coming from a pre-synaptic spike and run for 500 ms.

In [ ]:
model_params_e = {
                # Leak potential of the neurons (in mV).
                # See Allen Cells GLIF Parameters.ipynb
                'E_L': -70.0,
                # Threshold potential of the neurons (in mV).
                # See Allen Cells GLIF Parameters.ipynb
                'V_th': -45.0,
                # Membrane potential after a spike (in mV).
                # See Allen Cells GLIF Parameters.ipynb
                'V_reset': -70.0,
                # Membrane capacitance (in pF).
                # See Allen Cells GLIF Parameters.ipynb
                'C_m': 300.0,
                # Membrane time constant (in ms).
                # See Allen Cells GLIF Parameters.ipynb
                # Lowered to account for high-conductance state.
                'tau_m': 10.0,
                # Time constant of postsynaptic excitatory currents (in ms).
                # Value for AMPA receptors from (Fourcaud & Brunel, 2002)
                'tau_syn_exc_AMPA': 2.0,
                # Value for NMDA receptors
                'tau_syn_exc_NMDA': 100.0,
                # Time constant of postsynaptic inhibitory currents (in ms).
                # Value for GABA_A receptors from (Fourcaud & Brunel, 2002)
                'tau_syn_inh': 5.0,
                # Refractory period of the neurons after a spike (in ms).
                't_ref': 2.0,
                'r_NMDA': 2.0,
                }

In [ ]:
nest.set_verbosity("M_WARNING")
nest.ResetKernel()

neuron = nest.Create(neuron_model_name)
nest.SetStatus(neuron, model_params_e)
voltmeter = nest.Create("voltmeter")

voltmeter.set({"record_from": ["V_m"]})
nest.Connect(voltmeter, neuron)

sg = nest.Create("spike_generator", params={"spike_times": [20.]})
nest.Connect(sg, neuron, syn_spec={"receptor_type": 1, "weight": 100., "delay": 0.1})

sr = nest.Create("spike_recorder")
nest.Connect(neuron, sr)

nest.Simulate(500.)

spike_times = nest.GetStatus(sr, keys='events')[0]['times']

fig, ax = plt.subplots(nrows=1)
ax.plot(voltmeter.get("events")["times"], voltmeter.get("events")["V_m"])
ax.scatter(spike_times, 30 * np.ones_like(spike_times), marker="d", c="orange", alpha=.8, zorder=99)
ax.grid(True)
ax.set_ylabel("V [mV]")
ax.set_xlabel("Time [ms]")

In [ ]:
nest.GetStatus(nest.GetConnections(target=neuron, source=sg))

In [ ]:
NEST_SIMULATOR_INSTALL_LOCATION